# Part 3: VGG on MINST

In [15]:
import datetime
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D 
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras import datasets
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical


In [16]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [17]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

We can check the shape of new data and see that our images are 28×28 pixels, so we need to add a new axis, which will represent a number of channels. Also, it is important to do one-hot encoding of labels and normalization of input images.

In [18]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train[0].shape, 'image shape')

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples
(28, 28) image shape


In [19]:
# Add a new axis
x_train = x_train[:, :, :, np.newaxis]
x_test = x_test[:, :, :, np.newaxis]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(x_train[0].shape, 'image shape')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(28, 28, 1) image shape


In [20]:
# Convert class vectors to binary class matrices.

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [21]:
# Data normalization
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [22]:
#define VGG
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 7, 7, 128)        

In [23]:
adam = Adam(lr=1e-4, decay=1e-6)
#model.compile(optimizer=adam, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])

After creating a model, we need to train its parameters to make it powerful. Let’s train the model for a given number of epochs.


In [24]:
# Place the logs in a timestamped subdirectory
# This allows to easy select different training runs
# In order not to overwrite some data, it is useful to have a name with a timestamp
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Specify the callback object
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# tf.keras.callback.TensorBoard ensures that logs are created and stored
# We need to pass callback object to the fit method
# The way to do this is by passing the list of callback objects, which is in our case just one

In [25]:
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test),callbacks=[tensorboard_callback])

Epoch 1/20
  1/938 [..............................] - ETA: 0s - loss: 2.3027 - accuracy: 0.0781WARNING:tensorflow:From /Users/zhangmaoyu/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
938/938 [==============================] - 125s 133ms/step - loss: 0.7659 - accuracy: 0.7156 - val_loss: 0.5071 - val_accuracy: 0.8104
Epoch 2/20
938/938 [==============================] - 121s 129ms/step - loss: 0.4293 - accuracy: 0.8428 - val_loss: 0.4141 - val_accuracy: 0.8419
Epoch 3/20
938/938 [==============================] - 126s 134ms/step - loss: 0.3432 - accuracy: 0.8728 - val_loss: 0.3473 - val_accuracy: 0.8695
Epoch 4/20
938/938 [==============================] - 125s 133ms/step - loss: 0.3054 - accuracy: 0.8882 - val_loss: 0.3106 - val_accuracy: 0.8839
Epoch 5/20
938/938 [=============

In [26]:
%tensorboard --logdir=logs/fit

Reusing TensorBoard on port 6008 (pid 30593), started 0:55:02 ago. (Use '!kill 30593' to kill it.)

From the above results, it can be found that VGG performs very well on MINST data. Compared with Lenet, I think VGG will train better,since the training accuracy is up to 97%,and testing accuracy is about to 92%.and also I think the accuracy will be higher with the increase of the epoches,so that I hold the view that VGG performs better than Lenet on the MINST data set.

Next,we Train the model again with smaller learning rate to see the change of the loss and accuracy.

## Training the model again with smaller learning rate

In [27]:
adam = Adam(lr=1e-5, decay=1e-6)
model.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test),callbacks=[tensorboard_callback])

Epoch 1/20
938/938 [==============================] - 121s 129ms/step - loss: 0.0443 - accuracy: 0.9848 - val_loss: 0.2785 - val_accuracy: 0.9265
Epoch 2/20
938/938 [==============================] - 121s 129ms/step - loss: 0.0349 - accuracy: 0.9882 - val_loss: 0.2872 - val_accuracy: 0.9263
Epoch 3/20
938/938 [==============================] - 122s 130ms/step - loss: 0.0307 - accuracy: 0.9897 - val_loss: 0.3050 - val_accuracy: 0.9262
Epoch 4/20
938/938 [==============================] - 122s 130ms/step - loss: 0.0275 - accuracy: 0.9907 - val_loss: 0.3157 - val_accuracy: 0.9254
Epoch 5/20
938/938 [==============================] - 121s 129ms/step - loss: 0.0244 - accuracy: 0.9919 - val_loss: 0.3208 - val_accuracy: 0.9273
Epoch 6/20
938/938 [==============================] - 122s 130ms/step - loss: 0.0221 - accuracy: 0.9926 - val_loss: 0.3350 - val_accuracy: 0.9250
Epoch 7/20
938/938 [==============================] - 122s 130ms/step - loss: 0.0199 - accuracy: 0.9936 - val_loss: 0.3449 -

In [29]:
%tensorboard --logdir=logs/fit

Reusing TensorBoard on port 6008 (pid 30593), started 1:43:09 ago. (Use '!kill 30593' to kill it.)

When we train the model again with smaller learning rate,We can achieve optimal accuracy much faster，and the training accuracy can almost up to 100%,and the testing accuracy is larger than the larger learning rate in the fisrt part,the training loss is become smaller, but the validation loss is much larger than  the larger learning rate in the fisrt part.At some point, I think there might be an overfitting situation. Therefore,we can conclude that the VGG with perfect hyper-parameter (which is a mannual process )can perform much better than Lenet on MINST data，However, we also need to pay attention to the possibility that the model may overfit.